In [220]:
!java -version

openjdk version "11.0.23" 2024-04-16
OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [221]:
!pip install -q tabula-py
!pip install PyMuPDF
!pip install selenium requests beautifulsoup4

In [222]:
import os
import re
import tabula
import pandas as pd
import numpy as np
import fitz  # module from PyMuPDF

In [223]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## **Importing**

To download the files from the webpage (https://company.meralco.com.ph/news-and-advisories/rates-archives), we have to interact with the webpage and click the "Show more" button until it no longer appears. This can be achieved through selenium.

We can then extract all rows that contain "Summary Schedule of Rates" or "Summary of Schedule of Rates". We also obtain the date from the same row, which will be used in renaming the file. Lastly, we find the pdf link and download the pdf.

In [224]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import re
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

try:
    driver.get("https://company.meralco.com.ph/news-and-advisories/rates-archives")

    # Click "Show more" until all items are loaded
    while True:
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@class='btn btn-bordered load-more' and @title='Go to next page']"))
            )
            show_more_button.click()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            break  # Break the loop if no more "Show more" button is found

    rows = driver.find_elements(By.XPATH, "//tr[.//span[contains(text(), 'Summary Schedule of Rates') or contains(text(), 'Summary of Schedule of Rates')]]")

    # Prepare the download directory
    os.makedirs('downloads', exist_ok=True)

    for row in rows:
        try:
            # Find the date in the same row
            date_element = row.find_element(By.XPATH, ".//td[contains(@class, 'views-field-field-date-created')]")
            date_text = date_element.text.strip()

            # Convert date text to "mm-yy" format
            match = re.search(r'(\w+)\s+(\d{4})', date_text)
            if match:
                month_str, year_str = match.groups()
                month = {
                    'January': '01', 'February': '02', 'March': '03', 'April': '04',
                    'May': '05', 'June': '06', 'July': '07', 'August': '08',
                    'September': '09', 'October': '10', 'November': '11', 'December': '12'
                }[month_str]
                date_formatted = f"{month}-{year_str[-2:]}"

                # Break the loop if the date is August 2012
                if month_str == 'August' and year_str == '2012':
                    print("Reached August 2012. Stopping downloads.")
                    break
            else:
                print(f"Date format not recognized for text: {date_text}")
                continue  # Skip this row if date format is not recognized

            # Find the PDF link
            pdf_link = row.find_element(By.XPATH, ".//a[contains(@class, 'btn-bordered-orange') and contains(@href, '.pdf')]")
            url = pdf_link.get_attribute('href')

            # Download the PDF
            response = requests.get(url)
            if response.status_code == 200:
                pdf_name = f"Summary_Schedule_of_Rates_{date_formatted}.pdf"
                with open(f'downloads/{pdf_name}', 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Failed to download PDF from {url}, status code: {response.status_code}")

        except Exception as e:
            print(f"Error processing row: {e}")

finally:
    driver.quit()

Reached August 2012. Stopping downloads.


Do not import files before August 2012 as the succeeding code requires that the pdf have borders, which these files do not have.

**Missing Files**

* 02-17 (not on the webpage)

Since tabula cannot detect merged columns, we manually define the main headers and corresponding subheaders.

In [225]:
headers_and_subheaders = {
    "Customer Subclass": "NaN",
    "Generation Charge": "per kWh",
    "Prev. Mos' Adj on Gen": "per kWh",
    "Power Act Reduction": "per kWh",
    "Transmission Charge": "per kWh, per kW",
    "System Loss Charge": "per kWh",
    "Distribution Charge": "per kWh, per kW",
    "Supply Charge": "per kWh, per cust/mo",
    "Metering Charge": "per kWh, per cust/mo",
    "Reset Cost Adjustment": "per kWh",
    "Distribution Rate True-Up": "per kWh",
    "Dist Rate True-Up 1": "per kWh",
    "Dist Rate True-Up 2": "per kWh",
    "Dist Rate True-Up 3": "per kWh",
    "Dist Rate True-Up 4": "per kWh",
    "For Non-lifeline: Lifeline Subsidy": "per kWh",
    "For Lifeline: Applicable Discounts": "discount %",
    "Lifeline Rate Subsidy": "per kWh",
    "Senior Citizen Subsidy": "per kWh",
    "Current RPT Charge": "per kWh",
    "UC-ME":"per kWh",
    "UC-ME for (NPC-SPUG)": "per kWh",
    "UC-ME for (RED-CI)": "per kWh",
    "UC-EC": "per kWh",
    "UC-SCC": "per kWh",
    "UC-SD": "per kWh",
    "Fit-All (Renewable)": "per kWh",
    "Cross Subsidy Charge": "per kWh",
    "Current RPT": "per kWh",
    "Lifeline Discount": "%",
    "Special Discount": "%",
    "Power Factor Adj": "Penalty, Disc",
    "Prev. Mos' Adj on Gen Cost": "per kWh"
}

In [226]:
downloads_dir = "/content/downloads"

dataframes = {}

pdf_files = [f for f in os.listdir(downloads_dir) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    date_part = pdf_file.split('_')[-1].replace('.pdf', '')
    pdf_path = os.path.join(downloads_dir, pdf_file)

    df = tabula.read_pdf(pdf_path, lattice=True, pages=1)[0]

    dataframes[f"df_{date_part}"] = df

## **Processing**

### Processing Column Headers

In [227]:
def clean_column_name(col):
  col = col.replace('\r', ' ')  # Replace '\r' with a space
  col = re.sub(r'\s+', ' ', col)  # Replace multiple spaces with a single space
  # Special handling for "Dist Rate True-Up"
  if any(rate in col for rate in ['Dist Rate True-Up', 'Distribution Rate True-Up']):
      # Replace only '\r' and extra spaces, keep digits
      col = re.sub(r'[^\w\s\d-]', '', col)

  else:
      col = re.sub(r'\s*\d+\s*', ' ', col) # Remove superscripts

  return col.strip()  # Strip leading/trailing whitespace

In [228]:
def count_true_up_occurrences(columns):
    true_up_counter = 0
    for col in columns:
        if 'Up' in col:
            true_up_counter += 1
    return true_up_counter

In [229]:
def convert_column_names(columns):
    true_up_count = count_true_up_occurrences(columns)
    renamed_columns = []

    for col in columns:
        if 'Up' in col:
            if true_up_count == 1:
                renamed_columns.append("Distribution Rate True-Up")
            else:
                match = re.search(r'True-\s*Up\s*(\d)', col)
                if match:
                    num = match.group(1)
                    renamed_columns.append(f"Dist Rate True-Up {num}")
        else:
            renamed_columns.append(col)

    return renamed_columns

In [230]:
def process_headers(df):
  df.columns = convert_column_names(df.columns)
  df.columns = [clean_column_name(col) for col in df.columns]

  # Ensure unique column names by appending the DataFrame index if needed
  df.columns = [f"{col}_{i}" if col in df.columns[:i] else col for i, col in enumerate(df.columns)]

  # Clean the first row in a similar way and assign it back to the DataFrame
  df.iloc[0] = [clean_column_name(str(value)) for value in df.iloc[0]]

  non_null_values = df.iloc[0].dropna().values
  non_null_values = [val for val in non_null_values if val != 'nan']

  if "Universal Charge" in df.columns:
      universal_charge_index = df.columns.get_loc("Universal Charge")

      # Replace "Universal Charge" with the first non-null value
      df.columns.values[universal_charge_index] = non_null_values[0]

      # Insert any additional non-null values into the column headers
      remaining_values = non_null_values[1:]

      if remaining_values:
          # Split the columns into parts: before, at, and after "Universal Charge"
          before = df.columns[:universal_charge_index + 1]
          after = df.columns[universal_charge_index + 1:]

          # Create new columns with the remaining values inserted after the "Universal Charge" replacement
          new_columns = list(before) + remaining_values + list(after[:-len(remaining_values)])
          df.columns = new_columns

  # Replace "Lifeline Eligibility" with "For Non-lifeline: Lifeline Subsidy" and insert "For Lifeline: Applicable Discounts"
  if "Lifeline Eligibility" in df.columns:
      lifeline_index = df.columns.get_loc("Lifeline Eligibility")
      df.columns.values[lifeline_index] = "For Non-lifeline: Lifeline Subsidy"

      df.columns = list(df.columns[:lifeline_index + 1]) + ["For Lifeline: Applicable Discounts"] + list(df.columns[lifeline_index + 1:-1])

  # Replace column containing "SUMMARY" with "Customer Subclass"
  summary_column = next((col for col in df.columns if "SUMMARY" in col), None)
  if summary_column:
      df.rename(columns={summary_column: "Customer Subclass"}, inplace=True)

  # Prepare lists for multiindex tuples
  multi_index_headers = []

  for col in df.columns:
    if col in headers_and_subheaders:
        subheader = headers_and_subheaders[col].split(", ")
        for sub in subheader:
            multi_index_headers.append((col, sub))

  multi_index = pd.MultiIndex.from_tuples(multi_index_headers, names=['Charge', 'Unit'])

  df.columns = multi_index

  return df

In [231]:
for key in dataframes:
    if key == "df_09-23":
        continue  # Skip this dataframe and move to the next key
    df = dataframes[key]
    process_headers(df)

In [232]:
def crop_dataframe(df):
  start_index = df[df.iloc[:, 0] == 'Residential'].index[0]
  end_index = df[df.iloc[:, 0] == '400 W HPS (or equivalent)'].index[0]

  df = df.iloc[start_index+1:end_index + 1]
  df.reset_index(drop=True, inplace=True)

  return df

In [233]:
cleaned_dataframes = []

for key in dataframes:
    if key == "df_09-23":
        continue  # Skip this dataframe and move to the next key
    df = dataframes[key]
    date_str = key.split('_')[1]

    df['Date'] = date_str

    df = crop_dataframe(df)
    cleaned_dataframes.append(df)

In [234]:
combined_df = pd.concat(cleaned_dataframes, ignore_index=True, join='outer')

In [235]:
combined_df.head()

Charge Customer Subclass Generation Charge Transmission Charge         \
Unit                 NaN           per kWh             per kWh per kW   
0            0 TO 20 KWH            4.4574              0.7515    NaN   
1           21 TO 50 KWH            4.4574              0.7515    NaN   
2           51 TO 70 KWH            4.4574              0.7515    NaN   
3          71 TO 100 KWH            4.4574              0.7515    NaN   
4         101 TO 200 KWH            4.4574              0.7515    NaN   

Charge Distribution Charge        Supply Charge             Metering Charge  \
Unit               per kWh per kW       per kWh per cust/mo         per kWh   
0                   1.0012    NaN        0.5085       16.73          0.3377   
1                   1.0012    NaN        0.5085       16.73          0.3377   
2                   1.0012    NaN        0.5085       16.73          0.3377   
3                   1.0012    NaN        0.5085       16.73          0.3377   
4                   1.0012    NaN        0.5085       16.73          0.3377   

Charge             System Loss Charge   UC-ME   UC-EC   UC-SD  \
Unit   per cust/mo            per kWh per kWh per kWh per kWh   
0             5.00             0.2983  0.1561    0.00  0.0428   
1             5.00             0.2983  0.1561    0.00  0.0428   
2             5.00             0.2983  0.1561    0.00  0.0428   
3             5.00             0.2983  0.1561    0.00  0.0428   
4             5.00             0.2983  0.1561    0.00  0.0428   

Charge Fit-All (Renewable) Lifeline Rate Subsidy Senior Citizen Subsidy  \
Unit               per kWh               per kWh                per kWh   
0                   0.0983                   NaN                    NaN   
1                   0.0983                   NaN                    NaN   
2                   0.0983                   NaN                    NaN   
3                   0.0983                   NaN                    NaN   
4                   0.0983                0.0816               (0.0002)   

Charge Lifeline Discount Special Discount Power Factor Adj        Date  \
Unit                   %                %          Penalty Disc          
0                100.00%              NaN              NaN  NaN  01-21   
1                 50.00%              NaN              NaN  NaN  01-21   
2                 35.00%              NaN              NaN  NaN  01-21   
3                 20.00%              NaN              NaN  NaN  01-21   
4                    NaN              NaN              NaN  NaN  01-21   

Charge Power Act Reduction Dist Rate True-Up 1 Dist Rate True-Up 2  \
Unit               per kWh             per kWh             per kWh   
0                      NaN                 NaN                 NaN   
1                      NaN                 NaN                 NaN   
2                      NaN                 NaN                 NaN   
3                      NaN                 NaN                 NaN   
4                      NaN                 NaN                 NaN   

Charge Dist Rate True-Up 3 Dist Rate True-Up 4 Current RPT Charge  \
Unit               per kWh             per kWh            per kWh   
0                      NaN                 NaN                NaN   
1                      NaN                 NaN                NaN   
2                      NaN                 NaN                NaN   
3                      NaN                 NaN                NaN   
4                      NaN                 NaN                NaN   

Charge UC-ME for (NPC-SPUG) UC-ME for (RED-CI) Distribution Rate True-Up  \
Unit                per kWh            per kWh                   per kWh   
0                       NaN                NaN                       NaN   
1                       NaN                NaN                       NaN   
2                       NaN                NaN                       NaN   
3                       NaN                NaN                       NaN   
4 

In [236]:
def is_valid_float(value):
  try:
      return not pd.isna(value) and float(value) is not None
  except ValueError:
      return False

is_valid_float_df = combined_df.applymap(is_valid_float)

# Keep rows where there is at least one valid float-representing string
df = combined_df[is_valid_float_df.any(axis=1)]

### Cleaning Customer Subclass

In [237]:
df[("Customer Subclass", "NaN")].value_counts()

(Customer Subclass, NaN)
201 TO 300 KWH                              282
301 TO 400 KWH                              282
OVER 400 KWH                                282
VERY LARGE AND EXTRA LARGE 34.5 KV          141
MEDIUM SECONDARY                            141
VERY LARGE 13.8 KV AND BELOW                141
LARGE 13.8 KV AND BELOW                     141
MEDIUM 13.8 KV AND BELOW                    141
LARGE SECONDARY                             141
0 TO 200 KWH                                141
General Service B                           141
LARGE 34.5 KV                               141
101 TO 200 KWH                              141
GHMSCI                                      141
400 W HPS (or equivalent)                   141
MEDIUM 34.5 KV                              141
21 TO 50 KWH                                140
0 TO 20 KWH                                 140
71 TO 100 KWH                               140
51 TO 70 KWH                                140
125 W Mercury, 

In [238]:
def clean_subclass(subclass):
    if isinstance(subclass, str):
        if subclass.endswith("14"):
            subclass = subclass[:-2]

        if "HPS" in subclass:
            subclass = subclass.replace(" (or equivalent)", "(or equivalent)")

        if subclass == "VERY LARGE AND EXTRA LARGE 115 KV/69 K":
            subclass = "VERY LARGE AND EXTRA LARGE 115 KV/69 KV"

    return subclass

df[("Customer Subclass", "NaN")] = df[("Customer Subclass", "NaN")].apply(clean_subclass)

df[("Customer Subclass", "NaN")].value_counts()

<ipython-input-238-905b7f4514c8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[("Customer Subclass", "NaN")] = df[("Customer Subclass", "NaN")].apply(clean_subclass)


(Customer Subclass, NaN)
201 TO 300 KWH                             282
301 TO 400 KWH                             282
OVER 400 KWH                               282
0 TO 20 KWH                                141
21 TO 50 KWH                               141
400 W HPS(or equivalent)                   141
400 W Mercury, 250 W HPS(or equivalent)    141
250 W Mercury, 150 W HPS(or equivalent)    141
125 W Mercury, 70 W HPS(or equivalent)     141
GHMSCI                                     141
VERY LARGE AND EXTRA LARGE 34.5 KV         141
LARGE 34.5 KV                              141
MEDIUM 34.5 KV                             141
VERY LARGE 13.8 KV AND BELOW               141
LARGE 13.8 KV AND BELOW                    141
MEDIUM 13.8 KV AND BELOW                   141
LARGE SECONDARY                            141
MEDIUM SECONDARY                           141
General Service B                          141
0 TO 200 KWH                               141
101 TO 200 KWH                     

### Adding Customer Class

In [239]:
def map_customer_class(group):
    class_mapping = {
        "Residential": ["0 TO 20 KWH","21 TO 50 KWH","51 TO 70 KWH",
                        "71 TO 100 KWH","101 TO 200 KWH","201 TO 300 KWH",
                        "301 TO 400 KWH","OVER 400 KWH"],
        "General Service A": ["0 TO 200 KWH","201 TO 300 KWH","301 TO 400 KWH", "OVER 400 KWH"],
        "General Service B": ["General Service B"],
        "General Power (GP) Secondary": ["MEDIUM SECONDARY", "LARGE SECONDARY", "VERY LARGE SECONDARY"],
        "GP 13.8 KV and below": ["MEDIUM 13.8 KV AND BELOW", "LARGE 13.8 KV AND BELOW", "VERY LARGE 13.8 KV AND BELOW"],
        "GP 34.5 KV": ["MEDIUM 34.5 KV", "LARGE 34.5 KV", "VERY LARGE AND EXTRA LARGE 34.5 KV"],
        "GP 115 KV / 69 KV": ["LARGE 115 KV/69 KV", "VERY LARGE AND EXTRA LARGE 115 KV/69 KV"],
        "GP 115 KV / 69 KV": ["GP 115 KV / 69 KV","LARGE 115 KV/69 KV","VERY LARGE AND EXTRA LARGE 115 KV/69 KV"],
        "GHMSCI": ["GHMSCI"]
    }

    customer_class = []
    lower_limit_demand = []
    upper_limit_demand = []
    residential_flag = True
    general_service_a_flag = False
    flat_streetlights_flag = False

    for subclass in group[("Customer Subclass", "NaN")]:
        assigned_class = None
        lower_limit = None
        upper_limit = None

        if flat_streetlights_flag:
            assigned_class = "FLAT STREETLIGHTS"
        elif subclass == "Per kWh" or subclass == "125 W Mercury, 70 W HPS(or equivalent)":
            assigned_class = "FLAT STREETLIGHTS"
            flat_streetlights_flag = True
        elif residential_flag:
            assigned_class = "Residential"
            if subclass == "OVER 400 KWH":
                residential_flag = False
                general_service_a_flag = True
        elif general_service_a_flag:
            assigned_class = "General Service A"
            if subclass == "OVER 400 KWH":
                general_service_a_flag = False
        else:
            for class_name, subclasses in class_mapping.items():
                if subclass in subclasses:
                    assigned_class = class_name
                    break

        # Set the lower and upper limit demand based on the subclass
        if subclass == "General Service B":
            lower_limit = 5
            upper_limit = 40
        elif "MEDIUM" in subclass:
            lower_limit = 40
            upper_limit = 200
        elif "LARGE" in subclass:
            lower_limit = 200
            upper_limit = 750
        elif "VERY LARGE" in subclass:
            lower_limit = 750
            upper_limit = 10000
        elif "EXTRA LARGE" in subclass:
            lower_limit = 10000
            upper_limit = np.nan

        customer_class.append(assigned_class if assigned_class else "Unknown")
        lower_limit_demand.append(lower_limit)
        upper_limit_demand.append(upper_limit)

    group["Customer Class"] = customer_class
    group["Lower Limit Demand"] = lower_limit_demand
    group["Upper Limit Demand"] = upper_limit_demand
    return group

# Apply the mapping function group by group based on the Date
df = df.groupby(('Date', '')).apply(map_customer_class)

df = df.drop(('Date', ''), axis=1)

cols = [("Customer Class", "")] + [("Customer Subclass", "NaN")] + [("Lower Limit Demand", ""), ("Upper Limit Demand", "")] + [col for col in df.columns if col not in [("Customer Class", ""), ("Customer Subclass", "NaN"), ("Lower Limit Demand", ""), ("Upper Limit Demand", "")]]
df = df[cols]

In [240]:
df.head()

Charge        Customer Class Customer Subclass Lower Limit Demand  \
Unit                                       NaN                      
(Date, )                                                            
01-13    2452    Residential       0 TO 20 KWH                NaN   
         2453    Residential      21 TO 50 KWH                NaN   
         2454    Residential      51 TO 70 KWH                NaN   
         2455    Residential     71 TO 100 KWH                NaN   
         2456    Residential    101 TO 200 KWH                NaN   

Charge        Upper Limit Demand Generation Charge Transmission Charge         \
Unit                                       per kWh             per kWh per kW   
(Date, )                                                                        
01-13    2452                NaN            5.7910              0.8449    NaN   
         2453                NaN            5.7910              0.8449    NaN   
         2454                NaN            5.7910              0.8449    NaN   
         2455                NaN            5.7910              0.8449    NaN   
         2456                NaN            5.7910              0.8449    NaN   

Charge        Distribution Charge        Supply Charge              \
Unit                      per kWh per kW       per kWh per cust/mo   
(Date, )                                                             
01-13    2452              1.1945    NaN        0.5950       19.57   
         2453              1.1945    NaN        0.5950       19.57   
         2454              1.1945    NaN        0.5950       19.57   
         2455              1.1945    NaN        0.5950       19.57   
         2456              1.1945    NaN        0.5950       19.57   

Charge        Metering Charge             System Loss Charge   UC-ME   UC-EC  \
Unit                  per kWh per cust/mo            per kWh per kWh per kWh   
(Date, )                                                                       
01-13    2452          0.4003        5.00             0.6558  0.1163  0.0025   
         2453          0.4003        5.00             0.6558  0.1163  0.0025   
         2454          0.4003        5.00             0.6558  0.1163  0.0025   
         2455          0.4003        5.00             0.6558  0.1163  0.0025   
         2456          0.4003        5.00             0.6558  0.1163  0.0025   

Charge          UC-SD Fit-All (Renewable) Lifeline Rate Subsidy  \
Unit          per kWh             per kWh               per kWh   
(Date, )                                                          
01-13    2452     NaN                 NaN                   NaN   
         2453     NaN                 NaN                   NaN   
         2454     NaN                 NaN                   NaN   
         2455     NaN                 NaN                   NaN   
         2456     NaN                 NaN                0.1333   

Charge        Senior Citizen Subsidy Lifeline Discount Special Discount  \
Unit                         per kWh                 %                %   
(Date, )                                                                  
01-13    2452                    NaN           100.00%              NaN   
         2453                    NaN            50.00%              NaN   
         2454                    NaN            35.00%              NaN   
         2455                    NaN            20.00%              NaN   
         2456                 0.0001               NaN              NaN   

Charge        Power Factor Adj      Power Act Reduction Dist Rate True-Up 1  \
Unit                   Penalty Disc             per kWh             per kWh   
(Date, )                                                                      
01-13    2452              NaN  NaN            (0.0011)                 NaN   
         2453              NaN  NaN            (0.0011)                 NaN   
         2454              NaN  NaN            (0.0011)        

### Adding Computed Columns

In [241]:
def convert_value(val):
    if pd.isna(val) or val == "nan":
        return np.nan
    if isinstance(val, str):
        if val.startswith('(') and val.endswith(')'):
            return -float(val[1:-1])
        if val.endswith('%'):
            return float(val.strip('%')) / 100
        try:
            return float(val)
        except ValueError:
            return val  # Leave non-numeric strings as is
    return val

numeric_df = df.applymap(convert_value)
numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')

exclude_cols_kwh = ['Transmission Charge', 'Distribution Charge', 'Customer Class', 'Customer Subclass']
exclude_cols_kw = exclude_cols_kwh

combined_kwh = numeric_df.loc[:, numeric_df.columns.get_level_values(1) == 'per kWh'].drop(columns=exclude_cols_kwh, level=0).sum(axis=1, skipna=True)
combined_kw = numeric_df.loc[:, numeric_df.columns.get_level_values(1) == 'per kW'].drop(columns=exclude_cols_kw, level=0).sum(axis=1)

df[('kWh Attributable', np.nan)] = combined_kwh
df[('kW Attributable', np.nan)] = combined_kw

In [242]:
df.head()

Charge        Customer Class Customer Subclass Lower Limit Demand  \
Unit                                       NaN                      
(Date, )                                                            
01-13    2452    Residential       0 TO 20 KWH                NaN   
         2453    Residential      21 TO 50 KWH                NaN   
         2454    Residential      51 TO 70 KWH                NaN   
         2455    Residential     71 TO 100 KWH                NaN   
         2456    Residential    101 TO 200 KWH                NaN   

Charge        Upper Limit Demand Generation Charge Transmission Charge         \
Unit                                       per kWh             per kWh per kW   
(Date, )                                                                        
01-13    2452                NaN            5.7910              0.8449    NaN   
         2453                NaN            5.7910              0.8449    NaN   
         2454                NaN            5.7910              0.8449    NaN   
         2455                NaN            5.7910              0.8449    NaN   
         2456                NaN            5.7910              0.8449    NaN   

Charge        Distribution Charge        Supply Charge              \
Unit                      per kWh per kW       per kWh per cust/mo   
(Date, )                                                             
01-13    2452              1.1945    NaN        0.5950       19.57   
         2453              1.1945    NaN        0.5950       19.57   
         2454              1.1945    NaN        0.5950       19.57   
         2455              1.1945    NaN        0.5950       19.57   
         2456              1.1945    NaN        0.5950       19.57   

Charge        Metering Charge             System Loss Charge   UC-ME   UC-EC  \
Unit                  per kWh per cust/mo            per kWh per kWh per kWh   
(Date, )                                                                       
01-13    2452          0.4003        5.00             0.6558  0.1163  0.0025   
         2453          0.4003        5.00             0.6558  0.1163  0.0025   
         2454          0.4003        5.00             0.6558  0.1163  0.0025   
         2455          0.4003        5.00             0.6558  0.1163  0.0025   
         2456          0.4003        5.00             0.6558  0.1163  0.0025   

Charge          UC-SD Fit-All (Renewable) Lifeline Rate Subsidy  \
Unit          per kWh             per kWh               per kWh   
(Date, )                                                          
01-13    2452     NaN                 NaN                   NaN   
         2453     NaN                 NaN                   NaN   
         2454     NaN                 NaN                   NaN   
         2455     NaN                 NaN                   NaN   
         2456     NaN                 NaN                0.1333   

Charge        Senior Citizen Subsidy Lifeline Discount Special Discount  \
Unit                         per kWh                 %                %   
(Date, )                                                                  
01-13    2452                    NaN           100.00%              NaN   
         2453                    NaN            50.00%              NaN   
         2454                    NaN            35.00%              NaN   
         2455                    NaN            20.00%              NaN   
         2456                 0.0001               NaN              NaN   

Charge        Power Factor Adj      Power Act Reduction Dist Rate True-Up 1  \
Unit                   Penalty Disc             per kWh             per kWh   
(Date, )                                                                      
01-13    2452              NaN  NaN            (0.0011)                 NaN   
         2453              NaN  NaN            (0.0011)                 NaN   
         2454              NaN  NaN            (0.0011)        

In [243]:
df.to_excel("/content/Scrape MERALCO_draft.xlsx", merge_cells=True)

**Ideas and Observations**

* Transmission Charge is always based on actual demand.
* If billed demand is greater than actual demand, distribution charge is based on billed demand.

